# Baseline model

In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import os
import sys
import nltk

from nltk.corpus import stopwords

# Caching stopwords
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('english'))

sys.path.append(os.path.abspath('../src'))
from fact_classification import *

## Importing data

In [41]:
df, df_crowdsourced, df_ground_truth = data_loading(local=True)

## Preparing for processing

### Preparing text

In [42]:
df.head()

,Sentence_id,Text,Speaker,Speaker_title,Speaker_party,File_id,Length,Line_number,Sentiment,Verdict
0,16,I think we've seen a deterioration of values.,George Bush,Vice President,REPUBLICAN,1988-09-25.txt,8,16,0.000000,-1
1,17,I think for a while as a nation we condoned th...,George Bush,Vice President,REPUBLICAN,1988-09-25.txt,16,17,-0.456018,-1
2,18,"For a while, as I recall, it even seems to me ...",George Bush,Vice President,REPUBLICAN,1988-09-25.txt,29,18,-0.805547,-1
3,19,"So we've seen a deterioration in values, and o...",George Bush,Vice President,REPUBLICAN,1988-09-25.txt,35,19,0.698942,-1
4,20,"We got away, we got into this feeling that val...",George Bush,Vice President,REPUBLICAN,1988-09-25.txt,15,20,0.000000,-1


## Split into test and train

According to the description of the task we shuold split the dataset into test and train based on year of debate. All debates before and including 2008 goes into train and more recent debates into test. (We could also consider making a validation set when we get closer to the end to have a final validation)

In [43]:
df_train, df_test = test_train_split(df)

### Create the tfid matrix for the text column

In [44]:
train_tfid, test_tfid, vocabulary = tfid(train=df_train.Text, test=df_test.Text, n_gram_range=1)

In [45]:
train_tfid

<18170x10641 sparse matrix of type '<class 'numpy.float64'>'
	with 277846 stored elements in Compressed Sparse Row format>

In [46]:
test_tfid

<5363x10641 sparse matrix of type '<class 'numpy.float64'>'
	with 70684 stored elements in Compressed Sparse Row format>

Looks good, we first fitted the vectorizer to the train set (so only words in the train set will be counted) and then transformed the test set using the same vectorizer. They have the same amount of columns which indicate it has been done correctly, keeping them sparse to save storage.

## Predict using standard models

The base model used is `RandomForestClassifier` from the scikit-learn package. We set the `class_weight` parameter to `balanced_subsample` and leave the rest of the parameters as default.

In [47]:
pred_train, pred_test = predict_it(train_tfid, df_train.Verdict, test_tfid)

## Scoring

The score of our base-model is ok, it is important to also consider the scores for individual classes because our data is so unbalanced.

In [48]:
df_score_test = score_it(df_test.Verdict, pred_test)
df_score_train = score_it(df_train.Verdict, pred_train)

In [49]:
df_score_train

,algorithm,features,p_NFS,p_UFS,p_CFS,p_wavg,r_NFS,r_UFS,r_CFS,r_wavg,f_NFS,f_UFS,f_CFS,f_wavg
0,RFC,W,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [50]:
df_score_test

,algorithm,features,p_NFS,p_UFS,p_CFS,p_wavg,r_NFS,r_UFS,r_CFS,r_wavg,f_NFS,f_UFS,f_CFS,f_wavg
0,RFC,W,0.67,0.6,0.81,0.7,0.99,0.06,0.23,0.68,0.8,0.11,0.35,0.6


A very simple random forest classifier based only on the text vectorized gives an accuracy of around 60 percent. We can clearly see that it is strongly overtrained. It will function as a minimum effort baseline model example.

## Proving the point of checking more than one accuracy measure and your data

Making a model which will have a high score but be completely useless. We can predict all -1 and get an average weighted f1score of nearly 50%.

In [51]:
pred_test = dummy_model(df_test.shape[0])
pred_train = dummy_model(df_train.shape[0])

In [52]:
df_score_test = pd.concat([
    df_score_test,
    score_it(
        df_test.Verdict,
        pred_test,
        algorithm='DUM',
        features='None')
])

df_score_train = pd.concat([
    df_score_train,
    score_it(
        df_train.Verdict,
        pred_train,
        algorithm='DUM',
        features='None')
])

In [53]:
df_score_test

,algorithm,features,p_NFS,p_UFS,p_CFS,p_wavg,r_NFS,r_UFS,r_CFS,r_wavg,f_NFS,f_UFS,f_CFS,f_wavg
0,RFC,W,0.67,0.6,0.81,0.70,0.99,0.06,0.23,0.68,0.80,0.11,0.35,0.60
0,DUM,None,0.62,0.0,0.00,0.38,1.00,0.00,0.00,0.62,0.76,0.00,0.00,0.47


In [54]:
df_score_train

,algorithm,features,p_NFS,p_UFS,p_CFS,p_wavg,r_NFS,r_UFS,r_CFS,r_wavg,f_NFS,f_UFS,f_CFS,f_wavg
0,RFC,W,1.00,1.0,1.0,1.00,1.0,1.0,1.0,1.00,1.0,1.0,1.0,1.00
0,DUM,None,0.67,0.0,0.0,0.44,1.0,0.0,0.0,0.67,0.8,0.0,0.0,0.53


In [55]:
# Save results to files
score_saving(df_score_train, df_score_test)

In [56]:
# Export df_score_test to latex format for use in report
to_latex(df_score_test)

\begin{tabular}{llrrrrrrrrrrrr}
\toprule
algorithm & features &  p\_NFS &  p\_UFS &  p\_CFS &  p\_wavg &  r\_NFS &  r\_UFS &  r\_CFS &  r\_wavg &  f\_NFS &  f\_UFS &  f\_CFS &  f\_wavg \\
\midrule
      RFC &        W &   0.67 &    0.6 &   0.81 &    0.70 &   0.99 &   0.06 &   0.23 &    0.68 &   0.80 &   0.11 &   0.35 &    0.60 \\
      DUM &     None &   0.62 &    0.0 &   0.00 &    0.38 &   1.00 &   0.00 &   0.00 &    0.62 &   0.76 &   0.00 &   0.00 &    0.47 \\
\bottomrule
\end{tabular}



/mnt/c/Users/torol/OneDrive - Universitetet i Stavanger/DAT550/fact_check_worthy_claim/src/fact_classification.py:103: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(index=False))
